https://qiita.com/seri28/items/0b19e1c3e1cc584f71a4

https://qiita.com/jun40vn/items/d8a1f71fae680589e05c

### マウント

In [1]:
from google.colab import drive  # google.colabモジュールからdriveをインポート

drive.mount("/content/drive")  # Google Driveをマウントする

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### モジュールインポート

In [2]:
import itertools
import warnings
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### ファイル読み込み

In [3]:
path = "drive/My Drive/Colab Notebooks/Titanic/"

df_train = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "test.csv")
### おまじない
df_train['Survived'] = np.where(df_train['Survived'] == 1, 0, 1)

### 合成

In [4]:
df_train_len = len(df_train)
df = pd.concat([df_train, df_test], axis=0, ignore_index=True)

### 特徴量エンジニアリング

In [5]:
# Name
df["Title"] = df["Name"].map(lambda x: x.split(", ")[1].split(". ")[0])
df["Title"].replace(["Mme", "Ms"], "Mrs", inplace=True)
df["Title"].replace(["Mlle"], "Miss", inplace=True)
df["Title"].replace(
    [
        "Capt",
        "Col",
        "Major",
        "Dr",
        "Rev",
        "Don",
        "Sir",
        "the Countess",
        "Lady",
        "Dona",
        "Jonkheer",
    ],
    "Rare",
    inplace=True,
)

df_train["Surname"] = df_train["Name"].map(lambda name: name.split(",")[0].strip())
df["Surname"] = df["Name"].map(lambda name: name.split(",")[0].strip())
df_train["FamilyGroup"] = df_train["Surname"].map(df["Surname"].value_counts())
Female_Child_Group = df_train.loc[
    (df_train["FamilyGroup"] >= 2)
    & ((df_train["Age"] <= 16) | (df_train["Sex"] == "female"))
]
Female_Child_Group = Female_Child_Group.groupby("Surname")["Survived"].mean()
Male_Adult_Group = df_train.loc[
    (df_train["FamilyGroup"] >= 2)
    & (df_train["Age"] > 16)
    & (df_train["Sex"] == "male")
]
Male_Adult_List = Male_Adult_Group.groupby("Surname")["Survived"].mean()

Dead_list = set(
    Female_Child_Group[Female_Child_Group.apply(lambda x: x == 1)].index
) | set(Male_Adult_List[Male_Adult_List.apply(lambda x: x == 1)].index)
Survived_list = set(
    Female_Child_Group[Female_Child_Group.apply(lambda x: x == 0)].index
) | set(Male_Adult_List[Male_Adult_List.apply(lambda x: x == 0)].index)

df["Dead_list"] = 0
df["Survived_list"] = 0

df.loc[df["Surname"].isin(Dead_list), "Dead_list"] = 1
df.loc[df["Surname"].isin(Survived_list), "Survived_list"] = 1

df["Last_name"] = df["Name"].apply(lambda x: x.split(",")[0])

# Name & Title
df["Family_survival"] = 0.5
for grp, grp_df in df.groupby(["Last_name", "Fare"]):

    if len(grp_df) != 1:
        for index, row in grp_df.iterrows():
            smax = grp_df.drop(index)["Survived"].max()
            smin = grp_df.drop(index)["Survived"].min()
            passID = row["PassengerId"]

            if smax == 1.0:
                df.loc[df["PassengerId"] == passID, "Family_survival"] = 1
            elif smin == 0.0:
                df.loc[df["PassengerId"] == passID, "Family_survival"] = 0
for grp, grp_df in df.groupby("Ticket"):
    if len(grp_df) != 1:
        for ind, row in grp_df.iterrows():
            if (row["Family_survival"] == 0) | (row["Family_survival"] == 0.5):
                smax = grp_df.drop(ind)["Survived"].max()
                smin = grp_df.drop(ind)["Survived"].min()
                passID = row["PassengerId"]
                if smax == 1.0:
                    df.loc[df["PassengerId"] == passID, "Family_survival"] = 1
                elif smin == 0.0:
                    df.loc[df["PassengerId"] == passID, "Family_survival"] = 0


# Ticket
num_ticket = df[df["Ticket"].str.match("[0-9]+")].copy()
num_ticket_index = num_ticket.index.values.tolist()
num_alpha_ticket = df.drop(num_ticket_index).copy()

num_ticket["Ticket"] = num_ticket["Ticket"].apply(lambda x: int(x))
num_ticket["Ticket_cat"] = 0
num_ticket.loc[
    (num_ticket["Ticket"] >= 100000) & (num_ticket["Ticket"] < 200000), "Ticket_cat"
] = 1
num_ticket.loc[
    (num_ticket["Ticket"] >= 200000) & (num_ticket["Ticket"] < 300000), "Ticket_cat"
] = 2
num_ticket.loc[(num_ticket["Ticket"] >= 300000), "Ticket_cat"] = 3

num_alpha_ticket["Ticket_cat"] = 4
num_alpha_ticket.loc[num_alpha_ticket["Ticket"].str.match("A.+"), "Ticket_cat"] = 5
num_alpha_ticket.loc[num_alpha_ticket["Ticket"].str.match("C.+"), "Ticket_cat"] = 6
num_alpha_ticket.loc[
    num_alpha_ticket["Ticket"].str.match("C\.*A\.*.+"), "Ticket_cat"
] = 7
num_alpha_ticket.loc[num_alpha_ticket["Ticket"].str.match("F\.C.+"), "Ticket_cat"] = 8
num_alpha_ticket.loc[num_alpha_ticket["Ticket"].str.match("PC.+"), "Ticket_cat"] = 9
num_alpha_ticket.loc[num_alpha_ticket["Ticket"].str.match("S\.+.+"), "Ticket_cat"] = 10
num_alpha_ticket.loc[num_alpha_ticket["Ticket"].str.match("SC.+"), "Ticket_cat"] = 11
num_alpha_ticket.loc[num_alpha_ticket["Ticket"].str.match("SOTON.+"), "Ticket_cat"] = 12
num_alpha_ticket.loc[num_alpha_ticket["Ticket"].str.match("STON.+"), "Ticket_cat"] = 13
num_alpha_ticket.loc[
    num_alpha_ticket["Ticket"].str.match("W\.*/C.+"), "Ticket_cat"
] = 14

df = pd.concat([num_ticket, num_alpha_ticket]).sort_values("PassengerId")

# Family
df["Family"] = df["SibSp"] + df["Parch"] + 1
df["Alone"] = df["Family"].apply(lambda x: 1 if x == 1 else 0)
df["Family_small"] = df["Family"].apply(lambda x: 1 if (2 <= x and x < 5) else 0)
df["Family_middle"] = df["Family"].apply(lambda x: 1 if (5 <= x < 8) else 0)
df["Family_big"] = df["Family"].apply(lambda x: 1 if (8 <= x) else 0)

# Fare
df["Fare"] = df["Fare"].fillna(df.query('Pclass==3 & Embarked=="S"')["Fare"].median())

filter_condition = (df["Pclass"] == 1) & (df["Fare"] == 0)
filtered_data = df[filter_condition]
pclass1_median_fare = df[df["Pclass"] == 1]["Fare"].median()
df.loc[filter_condition, "Fare"] = pclass1_median_fare
filter_condition = (df["Pclass"] == 2) & (df["Fare"] == 0)
filtered_data = df[filter_condition]
pclass1_median_fare = df[df["Pclass"] == 2]["Fare"].median()
df.loc[filter_condition, "Fare"] = pclass1_median_fare
filter_condition = (df["Pclass"] == 3) & (df["Fare"] == 0)
filtered_data = df[filter_condition]
pclass1_median_fare = df[df["Pclass"] == 3]["Fare"].median()
df.loc[filter_condition, "Fare"] = pclass1_median_fare
df["Fare_cat"] = pd.qcut(df["Fare"], 4, labels=False)

# Cabin
df["Cabin"].fillna("n", inplace=True)
df["Cabin"] = df["Cabin"].str[0]

# Embarked
df["Embarked"].fillna("S", inplace=True)

# ダミー化
df = pd.get_dummies(
    df,
    columns=[
        "Sex",
        "Pclass",
        "Title",
        "Ticket_cat",
        "Fare_cat",
        "Cabin",
        "Embarked",
        "Family_survival",
    ],
)

# ドロップ
df.drop(["Name", "Last_name", "Ticket", "Cabin_T"], axis=1, inplace=True)

In [6]:
df_predict = df[
    [
        "Age",
        "Alone",
        "Family_small",
        "Family_middle",
        "Family_big",
        "Sex_female",
        "Sex_male",
        "Pclass_1",
        "Pclass_2",
        "Pclass_3",
        "Title_Master",
        "Title_Miss",
        "Title_Mr",
        "Title_Mrs",
        "Title_Rare",
        "Ticket_cat_0",
        "Ticket_cat_1",
        "Ticket_cat_2",
        "Ticket_cat_3",
        "Ticket_cat_4",
        "Ticket_cat_5",
        "Ticket_cat_6",
        "Ticket_cat_7",
        "Ticket_cat_8",
        "Ticket_cat_9",
        "Ticket_cat_10",
        "Ticket_cat_11",
        "Ticket_cat_12",
        "Ticket_cat_13",
        "Ticket_cat_14",
        "Fare_cat_0",
        "Fare_cat_1",
        "Fare_cat_2",
        "Fare_cat_3",
        "Cabin_A",
        "Cabin_B",
        "Cabin_C",
        "Cabin_D",
        "Cabin_E",
        "Cabin_F",
        "Cabin_G",
        "Cabin_n",
        "Embarked_C",
        "Embarked_Q",
        "Embarked_S",
    ]
]
known_age = df_predict[df_predict.Age.notnull()].values
unknown_age = df_predict[df_predict.Age.isnull()].values

X = known_age[:, 1:]
y = known_age[:, 0]

rfr = RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1)
rfr.fit(X, y)
predictedAges = rfr.predict(unknown_age[:, 1::])

df.loc[df.Age.isnull(), "Age"] = predictedAges
df["Age_cat"] = pd.qcut(df["Age"], q=5, labels=False, precision=1)
df = pd.get_dummies(df, columns=["Age_cat"])

In [7]:
Sex_columns = ["Sex_male", "Sex_female"]
Pclass_columns = ["Pclass_1", "Pclass_2", "Pclass_3"]
Age_columns = ["Age_cat_0", "Age_cat_1", "Age_cat_2", "Age_cat_3", "Age_cat_4"]
Family_columns = ["Alone", "Family_small", "Family_middle", "Family_big"]
Survived_columns = ["Survived_list"]
Dead_columns = ["Dead_list"]


def AND(data_frame, columns_1, columns_2):
    new_columns = {}
    combinations = list(itertools.product(columns_1, columns_2))
    for combo in combinations:
        new_column_name = " AND ".join(combo)
        new_columns[new_column_name] = (
            data_frame[combo[0]].fillna(0) & data_frame[combo[1]].fillna(0)
        ).astype(int)
    return pd.DataFrame(new_columns)


df = pd.concat([df, AND(df, Sex_columns, Family_columns)], axis=1)
df = pd.concat([df, AND(df, Pclass_columns, Sex_columns)], axis=1)
df = pd.concat([df, AND(df, Age_columns, Sex_columns)], axis=1)

In [8]:
print(list(df.columns))

['PassengerId', 'Survived', 'Age', 'SibSp', 'Parch', 'Fare', 'Surname', 'Dead_list', 'Survived_list', 'Family', 'Alone', 'Family_small', 'Family_middle', 'Family_big', 'Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare', 'Ticket_cat_0', 'Ticket_cat_1', 'Ticket_cat_2', 'Ticket_cat_3', 'Ticket_cat_4', 'Ticket_cat_5', 'Ticket_cat_6', 'Ticket_cat_7', 'Ticket_cat_8', 'Ticket_cat_9', 'Ticket_cat_10', 'Ticket_cat_11', 'Ticket_cat_12', 'Ticket_cat_13', 'Ticket_cat_14', 'Fare_cat_0', 'Fare_cat_1', 'Fare_cat_2', 'Fare_cat_3', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_n', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Family_survival_0.0', 'Family_survival_0.5', 'Family_survival_1.0', 'Age_cat_0', 'Age_cat_1', 'Age_cat_2', 'Age_cat_3', 'Age_cat_4', 'Sex_male AND Alone', 'Sex_male AND Family_small', 'Sex_male AND Family_middle', 'Sex_male AND Family_big', 'Sex_female AND Alone', 'Sex

### データ加工

In [9]:
df_predict = df[
    [
        "Survived",
        "Dead_list",
        "Survived_list",
        "Alone",
        "Family_small",
        "Family_middle",
        "Family_big",
        "Sex_female",
        "Sex_male",
        "Pclass_1",
        "Pclass_2",
        "Pclass_3",
        "Ticket_cat_0",
        "Ticket_cat_1",
        "Ticket_cat_2",
        "Ticket_cat_3",
        "Ticket_cat_4",
        "Ticket_cat_5",
        "Ticket_cat_6",
        "Ticket_cat_7",
        "Ticket_cat_8",
        "Ticket_cat_9",
        "Ticket_cat_10",
        "Ticket_cat_11",
        "Ticket_cat_12",
        "Ticket_cat_13",
        "Ticket_cat_14",
        "Cabin_A",
        "Cabin_B",
        "Cabin_C",
        "Cabin_D",
        "Cabin_E",
        "Cabin_F",
        "Cabin_G",
        "Cabin_n",
        "Family_survival_0.0",
        "Family_survival_0.5",
        "Family_survival_1.0",
        "Age_cat_0",
        "Age_cat_1",
        "Age_cat_2",
        "Age_cat_3",
        "Age_cat_4",
        "Sex_male AND Alone",
        "Sex_male AND Family_small",
        "Sex_male AND Family_middle",
        "Sex_male AND Family_big",
        "Sex_female AND Alone",
        "Sex_female AND Family_small",
        "Sex_female AND Family_middle",
        "Sex_female AND Family_big",
        "Pclass_1 AND Sex_male",
        "Pclass_1 AND Sex_female",
        "Pclass_2 AND Sex_male",
        "Pclass_2 AND Sex_female",
        "Pclass_3 AND Sex_male",
        "Pclass_3 AND Sex_female",
        "Age_cat_0 AND Sex_male",
        "Age_cat_0 AND Sex_female",
        "Age_cat_1 AND Sex_male",
        "Age_cat_1 AND Sex_female",
        "Age_cat_2 AND Sex_male",
        "Age_cat_2 AND Sex_female",
        "Age_cat_3 AND Sex_male",
        "Age_cat_3 AND Sex_female",
        "Age_cat_4 AND Sex_male",
        "Age_cat_4 AND Sex_female",
    ]
]
train = df_predict.iloc[:df_train_len]
test = df_predict.iloc[df_train_len:]
X_train = train.drop(["Survived"], axis=1)
y_train = train["Survived"]
X_test = test.drop(["Survived"], axis=1)

### ランダムフォレスト

In [10]:
param_grid = {"max_depth": [6], "min_samples_leaf": [1, 2, 4]}

rfc_gs = GridSearchCV(
    RandomForestClassifier(n_estimators=100, criterion="entropy", n_jobs=-1, random_state=42),
    param_grid,
    cv=5,
)
rfc_gs.fit(X_train, y_train)

print("Best Parameters: {}".format(rfc_gs.best_params_))
print("CV Score: {}".format(round(rfc_gs.best_score_, 3)))
rfc_pred = rfc_gs.predict_proba(X_test)[:, 1]

Best Parameters: {'max_depth': 6, 'min_samples_leaf': 4}
CV Score: 0.864


In [11]:
df_test["Survived"] = np.where(rfc_pred >= 0.42, 0, 1)

In [12]:
print(list(df_test["Survived"]))

[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 

### 提出ファイル作成

In [13]:
submission = df_test[["PassengerId", "Survived"]]
submission.to_csv(
    "/content/drive/My Drive/Colab Notebooks/Titanic/submission.csv",
    index=False,
)
from google.colab import files

files.download(
    "/content/drive/My Drive/Colab Notebooks/Titanic/submission.csv"
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>